# 트랜스포머 디코더 구현하기
[Crimsonjoo 깃허브](https://github.com/crimsonjoo/Easy-Transformer)

이 노트북은 PyTorch를 사용하여 트랜스포머 모델의 디코더 부분을 구현하는 방법을 단계별로 설명합니다.

트랜스포머 모델은 주로 자연어 처리(NLP) 분야에서 사용되는 모델로, 여러 개의 인코더와 디코더 레이어로 구성됩니다.

해당 노트북에서는 디코더 부분만을 구현해보겠습니다.


&nbsp;

&nbsp;

## 1. 필요한 라이브러리 불러오기

먼저, 이 구현에 필요한 PyTorch 라이브러리를 불러옵니다.

In [ ]:
import torch
import math
from torch import nn
import torch.nn.functional as F

&nbsp;

&nbsp;

## 2. 디코더 구성 요소 구현하기

트랜스포머 디코더를 구현하기 전에, 디코더가 사용하는 핵심 구성 요소들을 먼저 구현합니다.

이 구성 요소들은 멀티 헤드 어텐션, 포지션 와이즈 피드포워드 네트워크, 그리고 레이어 노멀라이제이션입니다.


####**1) 스케일드 닷 프로덕트 어텐션 함수**

어텐션 메커니즘은 입력 시퀀스의 중요한 부분에 더 많은 주의를 기울이도록 모델을 돕습니다.


스케일드 닷 프로덕트 어텐션은 이러한 어텐션 메커니즘의 한 형태입니다.


In [ ]:
# 스케일드 닷 프로덕트 어텐션 함수
def scaled_dot_product(q, k, v, mask=None):
    d_k = q.size()[-1]  # 키 벡터의 차원
    scaled = torch.matmul(q, k.transpose(-1, -2)) / math.sqrt(d_k)  # 스케일링
    if mask is not None:
        scaled += mask  # 마스크가 있으면 추가
    attention = F.softmax(scaled, dim=-1)  # 소프트맥스를 통한 어텐션 가중치 계산
    values = torch.matmul(attention, v)  # 가중치를 값 벡터에 적용
    return values, attention

####**2) 멀티 헤드 어텐션 클래스**

멀티헤드 어텐션 클래스는 입력된 텍스트의 다양한 부분에 주의를 기울여 정보를 종합하는 데 도움을 줍니다.

이를 통해 모델이 더 많은 맥락을 이해할 수 있게 됩니다.



In [ ]:
# 멀티헤드 어텐션
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads
        assert self.head_dim * num_heads == d_model, "d_model의 차원 크기는 반드시 num_heads의 배수"
        self.qkv_layer = nn.Linear(d_model, 3 * d_model)
        self.linear_layer = nn.Linear(d_model, d_model)

    def forward(self, x, mask=None):
        batch_size, sequence_length, d_model = x.size()
        qkv = self.qkv_layer(x).view(batch_size, sequence_length, self.num_heads, 3 * self.head_dim)
        qkv = qkv.permute(0, 2, 1, 3).chunk(3, dim=-1)
        q, k, v = qkv
        values, attention = scaled_dot_product(q, k, v, mask)
        values = values.permute(0, 2, 1, 3).contiguous().view(batch_size, sequence_length, d_model)
        out = self.linear_layer(values)
        return out

####**3) 포지션 와이즈 피드포워드 네트워크**

이 네트워크는 각 위치에서 독립적으로 동작하는 전결합 레이어를 통해 정보를 처리합니다.



In [ ]:
# 위치별 전결합 피드포워드 네트워크
class PositionwiseFeedForward(nn.Module):
    def __init__(self, d_model, hidden, drop_prob=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.linear1 = nn.Linear(d_model, hidden)
        self.linear2 = nn.Linear(hidden, d_model)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=drop_prob)

    def forward(self, x):
        x = self.linear1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.linear2(x)
        return x

####**4) 레이어 정규화**

레이어 정규화는 각 레이어의 입력을 정규화하여 학습을 안정화시키는 기술입니다.



In [ ]:
# 레이어 정규화
class LayerNormalization(nn.Module):
    def __init__(self, parameters_shape, eps=1e-5):
        super().__init__()
        self.gamma = nn.Parameter(torch.ones(parameters_shape))
        self.beta = nn.Parameter(torch.zeros(parameters_shape))
        self.eps = eps

    def forward(self, inputs):
        mean = inputs.mean(dim=-1, keepdim=True)
        std = inputs.std(dim=-1, keepdim=True) + self.eps
        y = (inputs - mean) / std
        out = self.gamma * y + self.beta
        return out

####**5) 디코더 레이어 클래스**

디코더 레이어는 멀티 헤드 어텐션과 포지션 와이즈 피드포워드 네트워크를 포함합니다.

이 레이어는 입력 시퀀스를 변환하여 다음 레이어로 전달하는 역할을 합니다.



In [ ]:
# 디코더 레이어
class DecoderLayer(nn.Module):
    def __init__(self, d_model, ffn_hidden, num_heads, drop_prob):
        super(DecoderLayer, self).__init__()
        self.self_attention = MultiHeadAttention(d_model, num_heads)
        self.norm1 = LayerNormalization([d_model])
        self.dropout1 = nn.Dropout(p=drop_prob)

        self.ffn = PositionwiseFeedForward(d_model, ffn_hidden, drop_prob)
        self.norm2 = LayerNormalization([d_model])
        self.dropout2 = nn.Dropout(p=drop_prob)

    def forward(self, x, self_mask):
        attention = self.self_attention(x, mask=self_mask)
        x = self.norm1(x + self.dropout1(attention))
        ffn_output = self.ffn(x)
        x = self.norm2(x + self.dropout2(ffn_output))
        return x

####**6) 디코더 클래스**

이제 모든 구성 요소를 하나로 합쳐 디코더를 구성합니다.

디코더는 여러 개의 디코더 레이어를 포함하며, 각 레이어는 입력 데이터를 처리하여 최종 출력을 생성합니다.



In [ ]:
# 디코더 클래스
class Decoder(nn.Module):
    def __init__(self, d_model, ffn_hidden, num_heads, drop_prob, num_layers):
        super().__init__()
        self.layers = nn.ModuleList([DecoderLayer(d_model, ffn_hidden, num_heads, drop_prob) for _ in range(num_layers)])
        self.norm = LayerNormalization([d_model])

    def forward(self, x, self_mask):
        for layer in self.layers:
            x = layer(x, self_mask)
        x = self.norm(x)
        return x

&nbsp;

&nbsp;

## 3. 모델 실행

마지막으로, 디코더 모델을 초기화하고 입력 데이터에 대해 실행해보겠습니다.

이를 통해 모델이 정상적으로 작동하는지 확인할 수 있습니다.




In [ ]:
# 모델 config
d_model = 512
num_heads = 8
drop_prob = 0.1
batch_size = 30
max_sequence_length = 200
ffn_hidden = 2048
num_layers = 5


# 모델 선언
x = torch.randn( (batch_size, max_sequence_length, d_model) ) # English sentence positional encoded
y = torch.randn( (batch_size, max_sequence_length, d_model) ) # Korean sentence positional encoded
mask = torch.full([max_sequence_length, max_sequence_length] , float('-inf'))
mask = torch.triu(mask, diagonal=1)
decoder = Decoder(d_model, ffn_hidden, num_heads, drop_prob, num_layers)


# 모델 실행
out = decoder(x, y, mask)